# Calculate Collisons per second at sea level
#### Mean Free Path (λ): 6.45e-08 meters
#### Average Thermal Speed (v_avg): 466.53 meters/second
#### Number of Collisions per Second: 7.23e+09 collisions/second
#### time to collide: 1.5 e-10 seconds

In [ ]:
import math
# Constants
k_B = 1.38e-23; # Boltzmann constant in J/K
T = 288; # Temperature at sea level in Kelvin
d = 3.7e-10; # Diameter of a nitrogen molecule in meters
P = 101325; # Pressure at sea level in Pascals
m = 4.65e-26; # Mass of a nitrogen molecule in kg
pi = math.pi; # Value of Pi
# ====================================================================================================
# Number density of air molecules (n_air): 2.446e+25 molecules/m³
# Number density of CO₂ molecules (n_CO2): 9.783e+21 molecules/m³
# # Spectral radiance (B_lambda): 6.693e+06 W·m⁻²·sr⁻¹·m⁻¹
# Spectral photon flux density (N_lambda): 5.054e+26 photons·m⁻²·s⁻¹·m⁻¹
# Total photon flux (Phi_total): 5.054e+20 photons·m⁻²·s⁻¹
# Absorption rate per CO₂ molecule (R): 5.054e-02 s⁻¹
# Average time between photon absorptions (tau): 19.79 seconds

# Step 1: Mean free path calculation
def meanFreePath(k_B, T, d, P):
    return (k_B * T) / (math.sqrt(2) * pi * math.pow(d, 2) * P)


# Step 2: Thermal speed calculation
def thermalSpeed(k_B, T, m):
    return math.sqrt((8 * k_B * T) / (pi * m))

# Step 3: Number of collisions per second
def numberOfCollisions(v_avg, lamda):
    return v_avg / lamda

# Calculate the mean free path (lamda)
lamda = meanFreePath(k_B, T, d, P)

# Calculate the average thermal speed (v_avg)
v_avg = thermalSpeed(k_B, T, m)

# Calculate the number of collisions per second (Z)
collisionsPerSecond = numberOfCollisions(v_avg, lamda)

# Output the results
print(f'Mean Free Path (λ): {round(lamda,10)} meters')
print(f'Average Thermal Speed (v_avg): {round(v_avg,2)} meters/second')
print(f'Number of Collisions per Second: {'{:.2e}'.format(collisionsPerSecond)} collisions/second')
print(f'time to collide: {1/collisionsPerSecond} seconds')


Mean Free Path (λ): 6.45e-08 meters
Average Thermal Speed (v_avg): 466.53 meters/second
Number of Collisions per Second: 7.23e+09 collisions/second
time to collide: 1.38231581027077e-10 seconds
